In [786]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [787]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [788]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'
baseurl = 'http://127.0.0.1:7000/'

In [789]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [790]:
response.json()

{'signingKey_hex': '880cdbe26e71f80f5e867d6b35b23709e43f9d6b7ad24e9717795aa7bd7af2d7',
 'traderId': '3',
 'verifyKey_hex': 'fd159b3c56ac166208009dafe016c0eab98c2bac8d2b7be8b6ca211753da228d'}

In [791]:
url = baseurl + 'createMarket'

mkRtId = 6
mkId=1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [792]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkRtId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [793]:
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,6,3


In [794]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk,
                     "traderId": tId,
                     "verifyKey_hex": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}

In [795]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [796]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': 1,
 'traderId': '3'}

In [757]:
# View json response
pd.DataFrame(response.json(), index=[0])


,checks,marketId,price,quantity,traderId
0,True,1,0.55,1,1


In [758]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [759]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': -1,
 'traderId': '1'}

In [760]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.37020397186279297 seconds.
matched trade: 1 in 0.34033894538879395 seconds.
matched trade: 2 in 0.33739590644836426 seconds.
matched trade: 3 in 0.329373836517334 seconds.
matched trade: 4 in 0.32706403732299805 seconds.


In [761]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.25220203399658203 seconds.
market created: 2 in 0.3762812614440918 seconds.
market created: 3 in 0.6797239780426025 seconds.
market created: 4 in 1.4314167499542236 seconds.


In [762]:
tId

'1'

In [763]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 1}

In [764]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [765]:
a = pd.read_json(response.json())
a

,marketId,marketRootId,marketBranchId,marketMin,marketMax
0,1,6,1,0,1.0
1,2,1,1,0,0.7
2,3,2,1,0,0.7
3,4,3,1,0,0.7
4,5,4,1,0,0.7


In [766]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [767]:
a = pd.read_json(response.json())
a

,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,1,0.55,1,1,True,2019-01-20 03:03:23.540
1,1,0.55,-1,1,True,2019-01-20 03:03:28.342
10,1,0.55,-1,1,True,2019-01-20 03:03:59.916
11,1,0.55,1,1,True,2019-01-20 03:04:00.082
2,1,0.55,-1,1,True,2019-01-20 03:03:58.576
3,1,0.55,1,1,True,2019-01-20 03:03:58.747
4,1,0.55,-1,1,True,2019-01-20 03:03:58.913
5,1,0.55,1,1,True,2019-01-20 03:03:59.087
6,1,0.55,-1,1,True,2019-01-20 03:03:59.253
7,1,0.55,1,1,True,2019-01-20 03:03:59.425


In [768]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 0.43
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 1.2764179706573486 seconds.
unmatched trade: 1 in 1.356461763381958 seconds.
unmatched trade: 2 in 1.2543818950653076 seconds.
unmatched trade: 3 in 1.2466590404510498 seconds.
unmatched trade: 4 in 1.2640807628631592 seconds.


In [769]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.43,
 'quantity': -1,
 'traderId': '1'}

In [770]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [771]:
pd.read_json(response.json())

,marketId,price,quantity,traderId,timeStampUTC
12,1,0.43,-1,1,2019-01-20 03:04:21.054
13,1,0.43,-1,1,2019-01-20 03:04:22.410
14,1,0.43,-1,1,2019-01-20 03:04:23.666
15,1,0.43,-1,1,2019-01-20 03:04:24.912
16,1,0.43,-1,1,2019-01-20 03:04:26.176


In [772]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [773]:
pd.read_json(response.json())

,marketId,price,quantity,traderId,timeStampUTC
0,1,0.55,0,1,NaT


In [774]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 2.5586202144622803 seconds.
matched trade: 1 in 2.538774013519287 seconds.
matched trade: 2 in 2.502117872238159 seconds.
matched trade: 3 in 2.4970507621765137 seconds.
matched trade: 4 in 2.513331174850464 seconds.



# Curl 



## Create user

~~~~
curl --header "Content-Type: application/json" -X POST http://127.0.0.1:7000/createUser
~~~~
        
## Create market

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketRootId": 1, 
"marketBranchId": 5, 
"marketMin": 0, 
"marketMax": 1}' 
--header "Content-Type: application/json" -X POST http://localhost:7000/createMarket
~~~~
    
## Create trade

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketId": 1,
"price": 0.1212,
"quantity": 1}' --header "Content-Type: application/json" -X POST http://localhost:7000/createTrade

~~~~
        
## View order book (output to file)

~~~~
curl --header "Content-Type: application/json" -X POST http://127.0.0.1:7000/viewOrderBook --output ob.txt
~~~~    
        
Also can load json from file with something like 
~~~~ 
curl -d "@data.json" -X POST http://localhost:3000/data 
~~~~

Matlab will (probably?) run these with 
~~~~
[A, curlout] = system('curlstring')
~~~~

    

In [740]:
str(content_makemarket)


"{'signingKey_hex': '29ba39f7fea00b2639839d7740e875e884cdb593da4b46af8d4d9af3cc212bf0', 'traderId': 1, 'verifyKey_hex': 'e6376aef9f29b5ba13d98f388a2463d42e58300a4438545e769dbfa2a7432f07', 'marketRootId': 4, 'marketBranchId': 1, 'marketMin': 0, 'marketMax': 0.7}"

In [747]:
str(datetime.datetime.utcnow())

'2019-01-20 01:01:08.007049'